<a href="https://colab.research.google.com/github/bbutka/CEC220/blob/main/Binary_arithmetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

© 2024 Brian Butka. All rights reserved.

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

def binary_addition_with_carry(a, b):
    max_len = max(len(a), len(b))
    a = a.zfill(max_len)
    b = b.zfill(max_len)

    carry = 0
    result = ''
    carries = ''

    for i in range(max_len - 1, -1, -1):
        total = int(a[i]) + int(b[i]) + carry
        if total > 1:
            carry = 1
            carries = '1' + carries
        else:
            carry = 0
            carries = ' ' + carries  # Change '0' to ' ' for blank carry
        result = str(total % 2) + result

    # Shift carries one position to the left and add a trailing space
    carries = carries[1:] + ' '
    return result.zfill(max_len), carries.zfill(max_len), carry

def adjust_for_twos_complement(a, b):
    max_len = max(len(a), len(b))
    a = a.zfill(max_len)
    b = b.zfill(max_len)
    return a, b

def evaluate_twos_complement(bin_str):
    if bin_str[0] == '1':
        return int(bin_str, 2) - (1 << len(bin_str))
    else:
        return int(bin_str, 2)

def can_represent_in_bits(value, bits):
    min_val = -(1 << (bits - 1))
    max_val = (1 << (bits - 1)) - 1
    return min_val <= value <= max_val

def check_overflow(a, b, result, is_addition=True):
    if a[0] == b[0]:  # Same sign for addition
        if result[0] != a[0]:  # Different sign after operation
            return True
    if not is_addition and a[0] != b[0]:  # Different signs for subtraction
        if result[0] != a[0]:  # Different sign after operation
            return True
    return False

def binary_subtraction_with_steps(a, b, is_twos_complement=False):
    max_len = max(len(a), len(b))
    a = a.zfill(max_len)
    b = b.zfill(max_len)

    a_decimal = evaluate_twos_complement(a) if is_twos_complement else int(a, 2)
    b_decimal = evaluate_twos_complement(b) if is_twos_complement else int(b, 2)

    print(f"Subtraction Problem:")
    print(f"  {a} ({a_decimal})")
    print(f"- {b} ({b_decimal})")
    print(f"{'-' * (max_len + 2)}")

    # Function to handle borrowing
    def transform_borrow(a_list, i):
        j = i - 1
        while j >= 0 and a_list[j] == '0':
            a_list[j] = '1'
            j -= 1
        if j >= 0:
            a_list[j] = str(int(a_list[j]) - 1)
            for k in range(j + 1, i):
                a_list[k] = '1'
            a_list[i] = '2'  # Change the rightmost borrowing position to '2'
        else:
            a_list[i] = '2'
        #print(f"  Borrowing for position {i}: updated a = {''.join(a_list)}")
        return a_list

    # Step-by-step subtraction with borrow transformation
    a_list = list(a)
    b_list = list(b)
    result = []
    steps = []
    borrow = 0

    for i in range(max_len - 1, -1, -1):
        if int(a_list[i]) < int(b_list[i]):
            a_list = transform_borrow(a_list, i)
        sub = int(a_list[i]) - int(b_list[i]) - borrow
        if sub >= 2:
            borrow = 1
            sub -= 2
        else:
            borrow = 0
        result.insert(0, str(sub))

        # Record the current step
        current_step = {
            'step_number': max_len - i,
            'a': ''.join(a_list),
            'b': ''.join(b_list),
            'result': ''.join(result),
            'borrow': borrow
        }
        steps.append(current_step)

    # Print the final step
    final_step = steps[-1]
    final_result_decimal = evaluate_twos_complement(final_step['result']) if is_twos_complement else int(final_step['result'], 2)
    print(f"\nProblem with borrows")
    highlighted_a = ''.join(
        f"<span style='color: red; font-family: monospace; font-size: 14px;'>{final_step['a'][i]}</span>" if final_step['a'][i] != a[i] else f"<span style='font-family: monospace; font-size: 14px;'>{final_step['a'][i]}</span>"
        for i in range(len(a))
    )
    # Ensure proper alignment using non-breaking spaces
    spaces_before = '&nbsp;' * 4  # Adjust this number to align the output properly
    display(HTML(f"{spaces_before}{highlighted_a}&nbsp ({a_decimal})"))
    print(f"- {final_step['b']} ({b_decimal})")
    print(f"{'-' * (max_len + 2)}")
    print(f"  {' ' * (max_len - len(final_step['result']))}{final_step['result']} ({final_result_decimal})")

    final_result = ''.join(result)
    return final_result

def display_problem(operation, M, Q, result, carries, carry_out, carry_label, is_twos_complement):
    if is_twos_complement:
        M_dec = evaluate_twos_complement(M)
        Q_dec = evaluate_twos_complement(Q)
        result_dec = evaluate_twos_complement(result)
    else:
        M_dec = int(M, 2)
        Q_dec = int(Q, 2)
        if carry_out:
            result = '1' + result
        result_dec = int(result, 2)

    t = 0

    # Display the problem
    print("   ")
    print(f"\n{operation} Problem:")
    print(f"{' ' * (t+3)}{carries} ({carry_label})")
    print(f"{' ' * (t+1)}  {M} ({M_dec})")
    print(f"{' ' * (t+1)}{'+' if operation == 'Addition' else '-'} {Q} ({Q_dec})")
    print(f"{' ' * (t+1)}{'-' * (len(M) + 2)}")
    print(f"{' ' * (t+1-(carry_out*(1-is_twos_complement)))}  {result} ({result_dec})")
    if carry_out and is_twos_complement:
        print(f"Carry out: {carry_out}")

def main():
    def on_button_click(b):
        clear_output()
        main()

    binary_M = widgets.Text(
        value='',
        placeholder='Enter first binary number (up to 8 bits)',
        description='Binary M:',
        disabled=False
    )

    binary_Q = widgets.Text(
        value='',
        placeholder='Enter second binary number (up to 8 bits)',
        description='Binary Q:',
        disabled=False
    )

    is_twos_complement = widgets.RadioButtons(
        options=['Yes, 2\'s Complement', 'No, Binary'],
        description='2\'s Complement:',
        disabled=False
    )

    display(binary_M, binary_Q, is_twos_complement)

    def on_submit(button):
        M = binary_M.value
        Q = binary_Q.value
        is_twos = is_twos_complement.value == 'Yes, 2\'s Complement'

        # Adjust for 2's complement if needed
        if is_twos:
            M, Q = adjust_for_twos_complement(M, Q)

        # Perform binary addition
        sum_bin, sum_carries, sum_carry_out = binary_addition_with_carry(M, Q)

        # Check for overflow in addition for 2's complement
        if is_twos:
            if check_overflow(M, Q, sum_bin, is_addition=True):
                print("\nAddition Problem:")
                print("Overflow occurred, result cannot be represented in the given number of bits.")
            else:
                display_problem("Addition", M, Q, sum_bin, sum_carries, sum_carry_out, "carries", is_twos)
        else:
            display_problem("Addition", M, Q, sum_bin, sum_carries, sum_carry_out, "carries", is_twos)
        print("  ")
        # Perform binary subtraction
        if not is_twos and int(Q, 2) > int(M, 2):
            M_dec = int(M, 2)
            Q_dec = int(Q, 2)
            print("\nSubtraction Problem:")
            print(f"{M} ({M_dec}) - {Q} ({Q_dec}) - Negative Result")
        else:
            diff_bin = binary_subtraction_with_steps(M, Q, is_twos)
            if is_twos:
                diff_dec = evaluate_twos_complement(diff_bin)
                if not can_represent_in_bits(diff_dec, len(M)):
                    print("\nSubtraction Problem:")
                    print("Overflow occurred, result cannot be represented in the given number of bits.")
                elif check_overflow(M, Q, diff_bin, is_addition=False):
                    print("\nSubtraction Problem:")
                    print("Overflow occurred, result cannot be represented in the given number of bits.")

        display(restart_button)

    submit_button = widgets.Button(
        description='Submit',
        disabled=False,
        button_style='',
        tooltip='Click to submit',
        icon='check'
    )
    submit_button.on_click(on_submit)

    restart_button = widgets.Button(
        description='Restart',
        disabled=False,
        button_style='',
        tooltip='Click to restart',
        icon='refresh'
    )
    restart_button.on_click(on_button_click)

    display(submit_button)

main()


Text(value='', description='Binary M:', placeholder='Enter first binary number (up to 8 bits)')

Text(value='', description='Binary Q:', placeholder='Enter second binary number (up to 8 bits)')

RadioButtons(description="2's Complement:", options=("Yes, 2's Complement", 'No, Binary'), value="Yes, 2's Com…

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click to submit')


Addition Problem:
Overflow occurred, result cannot be represented in the given number of bits.

Subtraction Problem:
Overflow occurred, result cannot be represented in the given number of bits.


Button(description='Restart', icon='refresh', style=ButtonStyle(), tooltip='Click to restart')


Addition Problem:
                (carries)
          10000 (16)
        + 10000 (16)
        -------
         100000 (32)

Subtraction Problem:
                (borrows)
          10000 (16)
        - 10000 (16)
        -------
          00000 (0)


Button(description='Restart', icon='refresh', style=ButtonStyle(), tooltip='Click to restart')